In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from transformers import BertTokenizer
import gensim
import gensim.downloader

In [2]:
dataset = pd.read_excel("E:/domain/Synthetic User Stories.xlsx")


In [4]:
target = []
for row in dataset.iterrows():
    target.append(np.where(dataset["Domain"].unique() == row[1]["Domain"])[0][0])
dataset["Target"] = target
dataset["Target"]

0         0
1         1
2         0
3         1
4         0
         ..
12396    37
12397    36
12398    37
12399    36
12400    37
Name: Target, Length: 12401, dtype: int64

In [5]:
from transformers import RobertaTokenizer
from transformers import AlbertTokenizer
from transformers import DistilBertTokenizer

In [6]:
def getTrainSetFastText():
    ft_model = fasttext.load_model("fasttext_model.bin")
    traindata = []
    for msg in dataset['User Story']:
        traindata.append(ft_model.get_sentence_vector(msg))
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetTFIDF():
    countvec = CountVectorizer(max_features=100)
    bow = countvec.fit_transform(dataset['User Story']).toarray()
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(bow).toarray()
    training_data = pd.DataFrame(X)
    training_data.columns = training_data.columns.astype(str)
    return training_data

def getTrainSetBERT():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_data = tokenizer(dataset['User Story'].tolist(), padding=True, truncation=True, max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetRoBERTa():
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    tokenized_data = tokenizer(dataset['User Story'].tolist(), padding=True, truncation=True, max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetALBERT():
    tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
    tokenized_data = tokenizer(dataset['User Story'].tolist(), padding=True, truncation=True, max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetDistilBERT():
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    tokenized_data = tokenizer(dataset['User Story'].tolist(), padding=True, truncation=True, max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetWord2Vec():
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format('word2vec-google-news-300.bin', binary=True)
    traindata = []
    for msg in dataset['User Story']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in w2v_model:
                vecs.append(w2v_model[word][:100])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetGlove():
    glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')
    traindata = []
    for msg in dataset['User Story']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in glove_vectors:
                vecs.append(glove_vectors[word])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

In [12]:
pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/992.0 kB ? eta -:--:--
   --------------------- ------------------ 524.3/992.0 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 992.0/992.0 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetALBERT()  # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_rnn_model(input_length, vocab_size, num_classes):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128),  # Removed input_length
        LSTM(64, return_sequences=True),
        LSTM(32),
        Dense(num_classes, activation='softmax')  # Use softmax for multiclass classification
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_rnn_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)


    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "LSTM"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")


Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step
Mean Accuracy: 0.153
Mean Precision (Macro): 0.104
Mean Recall (Macro): 0.154
Mean F1-Score (Macro): 0.102
Mean Precision (Weighted): 0.105
Mean Recall (Weighted): 0.153
Mean F1-Score (Weighted): 0.103


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetALBERT()  # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_bilstm_model(input_length, vocab_size, num_classes, learning_rate=0.001, initializer=GlorotUniform()):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128, embeddings_initializer=initializer),  # Word embedding layer
        Bidirectional(LSTM(64, return_sequences=True, kernel_initializer=initializer)),  # BiLSTM layer 1
        Bidirectional(LSTM(32, kernel_initializer=initializer)),  # BiLSTM layer 2
        Dense(num_classes, activation='softmax', kernel_initializer=initializer)  # Output layer with softmax activation
    ])
    optimizer = Adam(learning_rate=learning_rate)  # Set the optimizer with learning rate
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_bilstm_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)


    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "LSTM"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")



Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step
Mean Accuracy: 0.993
Mean Precision (Macro): 0.957
Mean Recall (Macro): 0.958
Mean F1-Score (Macro): 0.957
Mean Precision (Weighted): 0.992
Mean Recall (Weighted): 0.993
Mean F1-Score (Weighted): 0.992


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetALBERT()  # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_gru_model(input_length, vocab_size, num_classes, learning_rate=0.001, initializer=GlorotUniform()):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128, embeddings_initializer=initializer),  # Word embedding layer
        GRU(64, return_sequences=True, kernel_initializer=initializer),  # GRU layer 1
        GRU(32, kernel_initializer=initializer),  # GRU layer 2
        Dense(num_classes, activation='softmax', kernel_initializer=initializer)  # Output layer with softmax activation
    ])
    optimizer = Adam(learning_rate=learning_rate)  # Set the optimizer with learning rate
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_gru_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)

    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "GRU"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")


Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step
Mean Accuracy: 0.990
Mean Precision (Macro): 0.954
Mean Recall (Macro): 0.955
Mean F1-Score (Macro): 0.954
Mean Precision (Weighted): 0.989
Mean Recall (Weighted): 0.990
Mean F1-Score (Weighted): 0.989


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetALBERT()  # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_bigru_model(input_length, vocab_size, num_classes, learning_rate=0.001, initializer=GlorotUniform()):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128, embeddings_initializer=initializer),  # Word embedding layer
        Bidirectional(GRU(64, return_sequences=True, kernel_initializer=initializer)),  # BiGRU layer 1
        Bidirectional(GRU(32, kernel_initializer=initializer)),  # BiGRU layer 2
        Dense(num_classes, activation='softmax', kernel_initializer=initializer)  # Output layer with softmax activation
    ])
    optimizer = Adam(learning_rate=learning_rate)  # Set the optimizer with learning rate
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_bigru_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)

    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "GRU"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")

Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
Mean Accuracy: 0.994
Mean Precision (Macro): 0.958
Mean Recall (Macro): 0.959
Mean F1-Score (Macro): 0.958
Mean Precision (Weighted): 0.993
Mean Recall (Weighted): 0.994
Mean F1-Score (Weighted): 0.993


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetALBERT()  # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_lstm_model(input_length, vocab_size, num_classes, learning_rate=0.001, initializer=GlorotUniform()):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128, embeddings_initializer=initializer),  # Word embedding layer
        LSTM(64, return_sequences=True, kernel_initializer=initializer),  # LSTM layer 1
        LSTM(32, kernel_initializer=initializer),  # LSTM layer 2
        Dense(num_classes, activation='softmax', kernel_initializer=initializer)  # Output layer with softmax activation
    ])
    optimizer = Adam(learning_rate=learning_rate)  # Set the optimizer with learning rate
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_lstm_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)

    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "GRU"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")

Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
Mean Accuracy: 0.116
Mean Precision (Macro): 0.069
Mean Recall (Macro): 0.121
Mean F1-Score (Macro): 0.063
Mean Precision (Weighted): 0.071
Mean Recall (Weighted): 0.116
Mean F1-Score (Weighted): 0.064


In [6]:
#Bilstm with roberta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetRoBERTa()  # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_bilstm_model(input_length, vocab_size, num_classes, learning_rate=0.001, initializer=GlorotUniform()):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128, embeddings_initializer=initializer),  # Word embedding layer
        Bidirectional(LSTM(64, return_sequences=True, kernel_initializer=initializer)),  # BiLSTM layer 1
        Bidirectional(LSTM(32, kernel_initializer=initializer)),  # BiLSTM layer 2
        Dense(num_classes, activation='softmax', kernel_initializer=initializer)  # Output layer with softmax activation
    ])
    optimizer = Adam(learning_rate=learning_rate)  # Set the optimizer with learning rate
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_bilstm_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)


    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "LSTM"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")

Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 90ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 92ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 92ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 93ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step
Mean Accuracy: 0.993
Mean Precision (Macro): 0.959
Mean Recall (Macro): 0.960
Mean F1-Score (Macro): 0.960
Mean Precision (Weighted): 0.992
Mean Recall (Weighted): 0.993
Mean F1-Score (Weighted): 0.992


In [6]:
# Bigru with roberta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetRoBERTa() # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_bigru_model(input_length, vocab_size, num_classes, learning_rate=0.001, initializer=GlorotUniform()):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128, embeddings_initializer=initializer),  # Word embedding layer
        Bidirectional(GRU(64, return_sequences=True, kernel_initializer=initializer)),  # BiGRU layer 1
        Bidirectional(GRU(32, kernel_initializer=initializer)),  # BiGRU layer 2
        Dense(num_classes, activation='softmax', kernel_initializer=initializer)  # Output layer with softmax activation
    ])
    optimizer = Adam(learning_rate=learning_rate)  # Set the optimizer with learning rate
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_bigru_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)

    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "GRU"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")

Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 109ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 93ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 94ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 9s 98ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 89ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 94ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step
Mean Accuracy: 0.993
Mean Precision (Macro): 0.955
Mean Recall (Macro): 0.954
Mean F1-Score (Macro): 0.954
Mean Precision (Weighted): 0.993
Mean Recall (Weighted): 0.993
Mean F1-Score (Weighted): 0.993


In [6]:
# GRU with roberta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetRoBERTa() # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_gru_model(input_length, vocab_size, num_classes, learning_rate=0.001, initializer=GlorotUniform()):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128, embeddings_initializer=initializer),  # Word embedding layer
        GRU(64, return_sequences=True, kernel_initializer=initializer),  # GRU layer 1
        GRU(32, kernel_initializer=initializer),  # GRU layer 2
        Dense(num_classes, activation='softmax', kernel_initializer=initializer)  # Output layer with softmax activation
    ])
    optimizer = Adam(learning_rate=learning_rate)  # Set the optimizer with learning rate
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_gru_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)

    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "GRU"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")

Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step
Mean Accuracy: 0.988
Mean Precision (Macro): 0.953
Mean Recall (Macro): 0.953
Mean F1-Score (Macro): 0.953
Mean Precision (Weighted): 0.987
Mean Recall (Weighted): 0.988
Mean F1-Score (Weighted): 0.988


In [6]:
#Bilstm with distilbert
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetDistilBERT()  # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_bilstm_model(input_length, vocab_size, num_classes, learning_rate=0.001, initializer=GlorotUniform()):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128, embeddings_initializer=initializer),  # Word embedding layer
        Bidirectional(LSTM(64, return_sequences=True, kernel_initializer=initializer)),  # BiLSTM layer 1
        Bidirectional(LSTM(32, kernel_initializer=initializer)),  # BiLSTM layer 2
        Dense(num_classes, activation='softmax', kernel_initializer=initializer)  # Output layer with softmax activation
    ])
    optimizer = Adam(learning_rate=learning_rate)  # Set the optimizer with learning rate
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_bilstm_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)


    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "LSTM"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")

Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step
Mean Accuracy: 0.994
Mean Precision (Macro): 0.958
Mean Recall (Macro): 0.959
Mean F1-Score (Macro): 0.958
Mean Precision (Weighted): 0.993
Mean Recall (Weighted): 0.994
Mean F1-Score (Weighted): 0.993


In [6]:
# Bigru with distilbert
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetDistilBERT() # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_bigru_model(input_length, vocab_size, num_classes, learning_rate=0.001, initializer=GlorotUniform()):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128, embeddings_initializer=initializer),  # Word embedding layer
        Bidirectional(GRU(64, return_sequences=True, kernel_initializer=initializer)),  # BiGRU layer 1
        Bidirectional(GRU(32, kernel_initializer=initializer)),  # BiGRU layer 2
        Dense(num_classes, activation='softmax', kernel_initializer=initializer)  # Output layer with softmax activation
    ])
    optimizer = Adam(learning_rate=learning_rate)  # Set the optimizer with learning rate
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_bigru_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)

    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "GRU"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")

Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 7s 115ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 102ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step
Mean Accuracy: 0.994
Mean Precision (Macro): 0.958
Mean Recall (Macro): 0.959
Mean F1-Score (Macro): 0.958
Mean Precision (Weighted): 0.993
Mean Recall (Weighted): 0.994
Mean F1-Score (Weighted): 0.993


In [7]:
# GRU with distilbert
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preparing the DataFrame to store results
result = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "Precision (Macro)", "Recall (Macro)", "F1-Score (Macro)",
                               "Precision (Weighted)", "Recall (Weighted)", "F1-Score (Weighted)"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)

# Assuming X is preprocessed sequence data, and y is the target labels
X = getTrainSetDistilBERT() # Replace with your sequence data processing
y = dataset['Target'].values  # Make sure this is your target (multiclass) column
counter = 0
foldcounter = 1

# Define lists to store each metric for mean calculation
accuracy_list = []
precision_macro_list = []
recall_macro_list = []
f1_macro_list = []
precision_weighted_list = []
recall_weighted_list = []
f1_weighted_list = []

# Define maximum sequence length for padding
max_sequence_length = 100  # Set appropriate length based on your data

def create_gru_model(input_length, vocab_size, num_classes, learning_rate=0.001, initializer=GlorotUniform()):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=128, embeddings_initializer=initializer),  # Word embedding layer
        GRU(64, return_sequences=True, kernel_initializer=initializer),  # GRU layer 1
        GRU(32, kernel_initializer=initializer),  # GRU layer 2
        Dense(num_classes, activation='softmax', kernel_initializer=initializer)  # Output layer with softmax activation
    ])
    optimizer = Adam(learning_rate=learning_rate)  # Set the optimizer with learning rate
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Find the maximum word index in your data
max_word_index = 0
for seq in X:
    seq_int = [int(x) for x in seq if str(x).isdigit()]
    if seq_int:
        max_word_index = max(max_word_index, max(seq_int))

# Cross-validation loop
for train_index, test_index in fold.split(X, y):
    print("Processing Fold "+ str(foldcounter) + " ...")

    # Split data into train and test sets for the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Convert y_train and y_test to one-hot encoding (for multiclass classification)
    y_train = np.eye(len(np.unique(y)))[y_train]  # One-hot encode labels
    y_test = np.eye(len(np.unique(y)))[y_test]  # One-hot encode labels

    # Convert X_train and X_test to lists of lists
    X_train = X_train.values.tolist()  # Convert DataFrame to list of lists
    X_test = X_test.values.tolist()  # Convert DataFrame to list of lists

    # Find the maximum word index in the sequences
    for seq in X_test:
        seq_int = [int(x) for x in seq if str(x).isdigit()]
        if seq_int:
            max_word_index = max(max_word_index, max(seq_int))

    # Add buffer to vocabulary size
    max_word_index += 50

    # Padding sequences to ensure consistent input shape
    X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length)

    # Create and train the model
    model = create_gru_model(input_length=max_sequence_length, vocab_size=max_word_index, num_classes=len(np.unique(y)))
    model.fit(X_train_padded, y_train, epochs=50, batch_size=256, verbose=0)

    # Make predictions on the test set (get the class with the highest probability)
    y_pred_prob = model.predict(X_test_padded)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels
    y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_test_labels, y_pred)
    precision_macro = precision_score(y_test_labels, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_test_labels, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
    
    # Calculate weighted metrics
    precision_weighted = precision_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_test_labels, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test_labels, y_pred, average='weighted', zero_division=0)

    # Store metrics in lists for mean calculation
    accuracy_list.append(accuracy)
    precision_macro_list.append(precision_macro)
    recall_macro_list.append(recall_macro)
    f1_macro_list.append(f1_macro)
    precision_weighted_list.append(precision_weighted)
    recall_weighted_list.append(recall_weighted)
    f1_weighted_list.append(f1_weighted)

    # Store the results in DataFrame
    result.loc[counter, "Fold"] = foldcounter
    result.loc[counter, "Model"] = "GRU"
    result.loc[counter, "Accuracy"] = round(accuracy, 3)
    result.loc[counter, "Precision (Macro)"] = round(precision_macro, 3)
    result.loc[counter, "Recall (Macro)"] = round(recall_macro, 3)
    result.loc[counter, "F1-Score (Macro)"] = round(f1_macro, 3)
    result.loc[counter, "Precision (Weighted)"] = round(precision_weighted, 3)
    result.loc[counter, "Recall (Weighted)"] = round(recall_weighted, 3)
    result.loc[counter, "F1-Score (Weighted)"] = round(f1_weighted, 3)

    counter += 1
    foldcounter += 1

# Calculate mean values for all metrics
mean_accuracy = np.mean(accuracy_list)
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)
mean_precision_weighted = np.mean(precision_weighted_list)
mean_recall_weighted = np.mean(recall_weighted_list)
mean_f1_weighted = np.mean(f1_weighted_list)

print(f"Mean Accuracy: {mean_accuracy:.3f}")
print(f"Mean Precision (Macro): {mean_precision_macro:.3f}")
print(f"Mean Recall (Macro): {mean_recall_macro:.3f}")
print(f"Mean F1-Score (Macro): {mean_f1_macro:.3f}")
print(f"Mean Precision (Weighted): {mean_precision_weighted:.3f}")
print(f"Mean Recall (Weighted): {mean_recall_weighted:.3f}")
print(f"Mean F1-Score (Weighted): {mean_f1_weighted:.3f}")

Processing Fold 1 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step
Processing Fold 2 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
Processing Fold 3 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step
Processing Fold 4 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step
Processing Fold 5 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step
Processing Fold 6 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step
Processing Fold 7 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step
Processing Fold 8 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step
Processing Fold 9 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step
Processing Fold 10 ...
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
Mean Accuracy: 0.988
Mean Precision (Macro): 0.953
Mean Recall (Macro): 0.954
Mean F1-Score (Macro): 0.953
Mean Precision (Weighted): 0.988
Mean Recall (Weighted): 0.988
Mean F1-Score (Weighted): 0.988
